In [ ]:
from MarkovNetwork import MarkovNetwork
import numpy as np
from tqdm import tqdm

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

creator.create('FitnessMulti', base.Fitness, weights=(1.0, -1.0))
creator.create('Individual', list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register('genome', np.random.randint, 0, 256, 10000)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.genome)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)

def eval_acc(individual):
    # Convert individual into MN
    acc = MarkovNetwork(num_input_states=9,
                        num_memory_states=10,
                        num_output_states=23,
                        probabilistic=False,
                        genome=individual)
    
    # Evaluate individual on digits
    return np.random.random(), np.random.random()

def pareto_selection_operator(individuals, k):
    return tools.selNSGA2(individuals, int(k / 5.)) * 5

def mutation_operator(individual):
    # Point mutations
    num_mutations = int(0.0005 * len(individual))
    for byte_index in np.random.randint(0, len(individual), num_mutations):
        individual[byte_index] = np.random.randint(0, 256)
    
    # Duplication
    if len(individual) < 50000 and np.random.random() < 0.05:
        width = np.random.randint(15, 601)
        start_index = np.random.randint(0, len(individual) - width)
        insert_index = np.random.randint(0, len(individual))
        for val in reversed(individual[start_index:start_index + width]):
            individual.insert(insert_index, val)
    
    # Deletion
    if len(individual) > 1000 and np.random.random() < 0.02:
        width = np.random.randint(15, 601)
        start_index = np.random.randint(0, len(individual) - width)
        del individual[start_index:start_index + width]
    
    return individual,


toolbox.register('evaluate', eval_acc)
toolbox.register('mutate', mutation_operator)
toolbox.register('select', pareto_selection_operator)

def pareto_eq(ind1, ind2):
    return np.all(ind1.fitness.values == ind2.fitness.values)

pop = toolbox.population(n=100)
hof = tools.ParetoFront(similar=pareto_eq)
stats = tools.Statistics(lambda ind: (int(ind.fitness.values[0]), round(ind.fitness.values[1], 2)))
stats.register('Minimum', np.min, axis=0)
stats.register('Maximum', np.max, axis=0)
stats.register('Progress', lambda x: pbar.update())

total_gens = 10

pbar = tqdm(total=total_gens)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0., mutpb=1.0, ngen=total_gens, 
                               stats=stats, halloffame=hof, verbose=False)
pbar.close()